In [ ]:
import numpy as np
import pandas as pd
import time
from ema_workbench.analysis import parcoords
from ema_workbench.analysis import clusterer, plotting, Density

from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from ema_workbench import save_results, load_results
from ema_workbench.analysis import clusterer, plotting, Density
import os
import matplotlib.image as mpimg
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import display, HTML
import dataframe_image as dfi
os.chdir(os.getcwd())
import sys

# insert at 1, 0 is the script path (or '' in REPL)
#pydice_folder = os.path.dirname(os.getcwd()) + "\\1_Model"
#sys.path.insert(1, pydice_folder)

from ema_workbench import (save_results)
from ema_workbench import (perform_experiments, Model, Policy, RealParameter, 
                           IntegerParameter, ScalarOutcome, ema_logging, MultiprocessingEvaluator)

ema_logging.log_to_stderr(ema_logging.INFO)

# Short term time series clustering

In [ ]:
timepoints_to_save = np.arange(2005,2105+10,10)

#objectives_list_timeseries_name = ['Damages ','Utility ',
#                        'Lowest income per capita ','Highest climate impact per capita ',
#                        'Distance to treshold ','Population under treshold ',
#                        'Intratemporal utility GINI ','Intratemporal impact GINI ',
#                        'Atmospheric Temperature ', 'Industrial Emission ', 'Total Output ']


objectives_list_timeseries_name =  ['Intratemporal utility GINI ','Intratemporal impact GINI ']

supplementary_list_timeseries_name = ['CPC ','Population ']
supplementary_list_quintile_name = ['CPC pre damage ','CPC post damage ']

outcomes = []
for name in objectives_list_timeseries_name:
    for year in timepoints_to_save:
        name_year = name + str(year)
        outcomes.append(name_year)

In [ ]:
timepoints_to_save = np.arange(2005,2105+10,10)                          
#objectives_list_timeseries_name =  {'egal':['Intratemporal utility GINI ','Intratemporal impact GINI '],
#                                    'util':['Utility ','Damages GINI '],
#                                    'prio':['Lowest income per capita','Highest climate impact per capita' ],
#                                    'suff':['Population under treshold','Distance to treshold',] }

objectives_list_timeseries_name =  ['Intratemporal utility GINI ','Intratemporal impact GINI ',
                                    'Utility ','Damages ','Lowest income per capita ',
                                    'Highest climate impact per capita ',
                                    'Population under treshold ','Distance to treshold ' ]

policy_list = ["Egal_policy36","Prio_policy28","Util_policy370","Suff_policy30"]

for index in range(0,2):
    policy_selected = policy_list[index]
    cluster_data_35k = results_total_short_term[results_total_short_term['policy_recoded'] == policy_selected]
    data = cluster_data_35k.to_numpy() 
    
    #randomly select 15000 timeseries for clustering
    rows = np.array(list(range(500)))
    np.random.seed(0)
    selected_series = np.random.choice(rows, size=500, replace=False)
    
    #get input data with select rows
    cluster_data_15k = data[selected_series]
    
    #construct original input dataframe with only rows from random selection (15k)
    input_data_15k = pd.DataFrame(data=cluster_data_15k,columns= cluster_data_35k.columns)
    input_data_15k = input_data_15k.set_index(input_data_15k.columns[0])
    
    #construct experiment dataframe with only scenario and policy columns
    experiments = input_data_15k.iloc[:,0:17]
    
    for objective in objectives_list_timeseries_name:
        outcomes = []
        for year in timepoints_to_save:
                name_year = objective + str(year)
                outcomes.append(name_year)

        data = input_data_15k[outcomes].to_numpy(dtype =float)
        #data = data.float()
                
        start = time.time()
        distances = clusterer.calculate_cid(data)
                
        #np.save('TSC_30k_scen_damages_distances', distances)
        end = time.time()
        print('Cluster time is ' + str(round((end - start))) + ' secondes')
        print("")
        
        #calculate silhouette width
        sil_score_lst = []
        start = time.time()
        for n_clusters in range(2,11):
            clusterers = AgglomerativeClustering(n_clusters=n_clusters, affinity='precomputed', linkage="complete")
            cluster_labels = clusterers.fit_predict(distances)
            silhouette_avg = silhouette_score(distances, cluster_labels, metric="precomputed")
            sil_score_lst.append(silhouette_avg)
            
            print(objective + "_" + policy_selected + ": For n_clusters =", n_clusters, "The average silhouette_score is :", silhouette_avg)
        
        end = time.time()
        print("")        
        print('Silhouette time is ' + str(round((end - start)/60)) + ' minutes')

        with open(objective + "_" + policy_selected + '_cluster_silhouette_width.txt', 'w') as f:
            for s in sil_score_lst:
                f.write(str(s) + "\n")
                
        #do agglomerative clustering on the distances
        start = time.time()
        for j in range(2, 8):
            clusters = clusterer.apply_agglomerative_clustering(distances,n_clusters=j)
            x = experiments.copy()
            x['clusters'] = clusters.astype('object')
            x.to_csv('TSC_35k_' + objective + "_" + policy_selected + '_cluster_' + str(j) + '.csv')
        end = time.time()
        print('Agglomerative clustering time is ' + str(round((end - start) / 60)) + ' minutes')
        print("")
        